# Stage 0: Crafting Effective System Prompts

Before we dive into building complex retrieval systems with our agent, we need to define who our agent is. To do so, we'll focus on the foundation of all context: system context.

By the end of this notebook, you will be able to:

1.  Define system context: Explore how to set an agent's role, capabilities, and constraints through system instructions.
2.  Evaluate system prompt effectiveness: You'll experiment with different system instructions and observe how they change agent behavior.

> ⚠️ **Note**: This notebook makes repeated calls to an LLM. The responses shown are examples captured when the notebook was authored, and your outputs may vary. We recommend running each query multiple times to assess both the system’s response consistency and the variety of replies.

Before diving in, let's recap what system context is and explore some industry-shared best practices before diving into crafting it with the Redis University agent. 

### The Art of System Context

To recap what we saw in the introduction to this course, system context (also known as a system prompt or system instructions) is the persistent context that defines your agent's identity and behavior. System context is typically included in every conversation turn you'll have with an LLM and tells it the following:

1.  Who it is (Role and identity)
2.  What it can do (Capabilities and tools)
3.  How it should behave (Personality and constraints)
4.  What it knows (Domain knowledge and context)

As you'll get to experience in this notebook and in your journey of building AI applications, setting system context can sometimes feel more of an art than a science. 

Anthropic shared some of their advice in what they find to be effective system prompts. In their guide on [Effective Context Engineering](https://www.anthropic.com/engineering/effective-context-engineering-for-ai-agents), Anthropic emphasizes that system prompts should be "extremely clear and use simple, direct language that presents ideas at the right altitude for the agent."

They describe this "right altitude" as a *Goldilocks Zone*:

> "At one extreme, we see engineers hardcoding complex, brittle logic in their prompts... At the other extreme, engineers sometimes provide vague, high-level guidance... The optimal altitude strikes a balance: specific enough to guide behavior effectively, yet flexible enough to provide the model with strong heuristics to guide behavior."

<img src="https://www.anthropic.com/_next/image?url=https%3A%2F%2Fwww-cdn.anthropic.com%2Fimages%2F4zrzovbb%2Fwebsite%2F0442fe138158e84ffce92bed1624dd09f37ac46f-2292x1288.png&w=3840&q=75" alt="Calibrating the system prompt" width="800"/>

*Image Source: Anthropic*

To achieve this balance, Anthropic recommends organizing prompts into distinct sections (like `<background_information>`, `<instructions>`, `## Tool guidance`, etc.) using XML tags or Markdown headers. 

In this notebook, we will apply a structured approach, and you'll see the difference a carefully crafted system prompt can make. Let's get started! 

## Setup

First, let's set up our environment. For this notebook, we're going to be using the Stage 1 baseline RAG agent as our foundation (located in the `/progressive_agents` directory). Don't worry about the internals yet — it's a basic retrieval pipeline that searches for courses and generates responses. We'll dive deep into how it works in the next notebook (stage 1).

The key thing to note is that we will be using a special function called `set_system_instructions`. This hook will allow us to dynamically override the agent's system prompt without modifying the source code — perfect for experimenting!

To begin, run the code blocks below to initialize the agent and load sample course data (50 courses) into Redis.

In [ ]:
#This code sets up the notebook to be able to access the provided OpenAI API Key and access to the agent code

import sys
import os
from pathlib import Path

if "OPENAI_API_BASE" in os.environ:
    os.environ["OPENAI_BASE_URL"] = os.environ["OPENAI_API_BASE"]

project_root = Path("..").resolve()

stage1_path = project_root / "progressive_agents" / "stage1_baseline_rag"
src_path = project_root / "src"

sys.path.insert(0, str(src_path))
sys.path.insert(0, str(stage1_path))

print('OpenAI API key and agent access setup!')

from agent import setup_agent
from agent.nodes import set_system_instructions

print("Initializing Stage 1 Baseline Agent...")
# We use the Stage 1 baseline agent, but we will override its system prompt
workflow, course_manager = setup_agent(auto_load_courses=True, verbose=False)
print("Agent is ready!")

## Crafting System Instructions

Let's build system instructions for our Redis University advisor agent step by step. We will override the agent's default system prompt and see how it changes the response.

### Example 1: Minimal System Instructions

Let's start with the bare minimum and see what happens. We're going to tell the system to be a "helpful assistant" but otherwise are not providing any meaningful guidance about its purpose and role. We'll test two queries: one about its identity and the other about what courses it would recommend.

Run the code block below to see the system's response. 

In [ ]:
# Minimal system prompt
minimal_prompt = "You are a helpful assistant."

# Override the agent's system instructions
set_system_instructions(minimal_prompt)

# Test the system's identity
print("Running agent query on identity...")
result_identity = await workflow.ainvoke({"query": "Who are you and what can you do?"})

print("Response on identity:")
print(result_identity['final_answer'])
print("\n" + "="*80 + "\n")

# Test the courses it will recommend
print("Running agent query on course recommendations...")
result_courses = await workflow.ainvoke({"query": "Give me a recommendation on a course I should take next semester"})

print("Response on course recommendations:")
print(result_courses['final_answer'])
print("\n" + "="*80 + "\n")

Since this application is an RAG, the LLM model is smart enough to recognize that we are working with course data (stored in our Redis database); however, in our testing (which may differ slightly from your own), we consistently received responses referencing that it was designed to help with business-specific courses: 

> "I am an AI assistant designed to help you with information about various courses offered in the Computer Science department"

And when asked for a specific course recommendation, it seemingly chooses a few courses at random related to math: 

> "If you are interested in machine learning and want to strengthen your mathematical foundation, **MATH022: Linear Algebra for Machine Learning** would be an excellent choice. If you are leaning towards computer science, **MATH029: Discrete Mathematics** is highly relevant. For a deeper dive into advanced mathematics, consider **MATH030: Abstract Algebra**, provided you meet the prerequisite. If you need to solidify your foundational skills, **MATH023: Pre-Calculus** is a great option."

With this in mind, we can identify two key improvements we want to make via the system prompt: 

1. We want it to identify itself properly and know exactly its purpose. This should steer it in the right direction, acting as a more general course advisor rather than one designed specifically for computer science. 

2. Secondly, we'll want it to have some level of consultative intelligence. We'll want the system to understand the nuance of a request, ask clarifying questions when necessary, and provide recommendations that actually make sense for the user's context.

Let's give it a try!

### Example 2: Adding Role and Purpose

Below, our system prompt has been upgraded. We explicitly define its identity and role. We will run it against the same set of queries from earlier, but in addition, we will also include a third query that poses an off-topic question. We will ask it to suggest a pizza recipe for a cooking class that the imaginary student is attending, and see how it responds. 

Run the code block below and observe the responses.

In [ ]:
# Add role and purpose
role_prompt = """You are the Redis University Class Agent.

Your role is to help students:
- Find courses that match their interests and requirements
- Plan their academic schedule
- Check prerequisites and eligibility
- Get personalized course recommendations
"""

# Override agent instructions
set_system_instructions(role_prompt)

queries = [
    {
        "label": "identity",
        "description": "Test the system's identity",
        "query": "Who are you and what can you do?"
    },
    {
        "label": "course recommendations",
        "description": "Test the courses it will recommend",
        "query": "Give me a recommendation on a course I should take next semester"
    },
    {
        "label": "off-topic query",
        "description": "Test an off-topic query",
        "query": "Suggest me a pizza recipe for my class"
    },
]

for q in queries:
    print(f"{q['description']}...")
    print(f"Running agent query on {q['label']}...")

    result = await workflow.ainvoke({"query": q["query"]})

    print(f"Response on {q['label']}:")
    print(result["final_answer"])
    print("\n" + "=" * 80 + "\n")

That response is much better! The system now understands its role and responds with a set of capabilities we have defined. In our testing for the first query, it consistently responded with its identity: 

> "I am the Redis University Class Agent, here to assist you with your academic journey. I can help you with the following: ..."

However, when asked for course recommendations, it seemingly chose a random set of courses related to math to recommend: 

> "Based on your request for a course recommendation for the next semester, I suggest considering the following options:

### 1. **Linear Algebra for Machine Learning (MATH022)** ..."

While we have resolved its identity issue, we still want the system to suggest courses thoughtfully, rather than randomly. This approach may resemble a more turn-based method, where the learner is consulted to learn about their preferences before a final suggestion is made. We'll implement a more complex version of the agent later in this course to handle multi-turn scenarios. 

Lastly, the system was presented with an off-topic question about making a pizza dough recipe. In our first attempt and several attempts after, we were able to get the system to return a recipe:

> "It seems like you're looking for a pizza recipe, but I specialize in helping with course information and academic planning. If you're interested in courses related to cooking or culinary arts, I can assist you in finding relevant classes. However, if you're looking for a pizza recipe, here's a simple one you can try: ..."

However, in good news, on several other attempts, it refused to return a recipe: 

> "It seems like you're looking for a pizza recipe, but I specialize in helping with course information and academic planning. If you're interested in courses related to cooking or culinary arts, I can assist you in finding relevant classes."

These types of inconsistencies can occur due to the nature of a model, but it's best practice to steer our system by providing specific guidelines on how to behave. For our third example around an off-topic query, let's add some behavioral guidance to improve the course suggestions and (hopefully) avoid returning an answer to an off-topic query.

### Example 3: Adding Behavioral Guidelines

In this next example, let's see how the system responds when presented with specific behavioral guidelines. 

Below, we extend the previous system prompt and will re-run the same queries as before. Our guidelines are designed to address the two core issues we observed in the earlier example: 

1. Providing more thorough assistance to the student
2. Ensuring the system does not respond to queries outside its domain.

To further test the preferences guideline, we have also added a fourth query requesting specific guidance on a topic the student mentioned. With a more specific query (e.g., “Learn about databases”), we would expect the system to recognize this as a preference and then search for courses related to databases.

Examine the code block below and then run it to see the response.

In [ ]:
# Add behavioral guidelines
guideline_prompt = """You are the Redis University Class Agent.

Your role is to help students:
- Find courses that match their interests and requirements
- Plan their academic schedule
- Check prerequisites and eligibility
- Get personalized course recommendations

Guidelines:
- Do not recommend any courses unless you know about the student's preferences
- If asked about topics outside your domain and responsibilities, always politely redirect to course planning
"""

# Override agent instructions
set_system_instructions(guideline_prompt)

queries = [
    {
        "label": "identity",
        "description": "Test the system's identity",
        "query": "Who are you and what can you do?"
    },
    {
        "label": "course recommendations",
        "description": "Test the courses it will recommend",
        "query": "Give me a recommendation on a course I should take next semester"
    },
    {
        "label": "off-topic query",
        "description": "Test an off-topic query",
        "query": "Suggest me a pizza recipe for my class"
    },
    {
        "label": "specific query",
        "description": "Test a specific query",
        "query": "I want to learn about databases and am not sure where to start"
    },
]

for q in queries:
    print(f"{q['description']}...")
    print(f"Running agent query on {q['label']}...")

    result = await workflow.ainvoke({"query": q["query"]})

    print(f"Response on {q['label']}:")
    print(result["final_answer"])
    print("\n" + "=" * 80 + "\n")

In our various tests at the time of authoring this notebook, the new guidelines provided to the system prompt sufficiently improved the system's response. 

When asked about course recommendations without knowing specific preferences, the system successfully utilized our guideline ("Do not recommend any courses unless you know about the student's preferences") to facilitate a more fluid conversation. It typically responded with the following: 

> "To provide you with a personalized course recommendation, I need to know more about your interests and academic goals. Here are a few questions to help narrow down the options: ..."

We were also unable to reproduce the issue with the system providing a pizza recipe, and it consistently tried to redirect based on the second guideline ("If asked about topics outside your domain and responsibilities, always politely redirect to course planning"). It typically responded with the following: 

> "I'm here to assist you with course planning and academic scheduling, but I can't provide recipes or cooking advice..."

Lastly, we confirmed that the guideline regarding learner preferences would also function properly when the query provided specifics (e.g., database courses). Our system typically returned a few recommendations related to the topic: 

> "If you're interested in learning about databases, I recommend considering the following course: Course: Database Systems (CS005) ..."

Overall, it was a solid upgrade to our system context and has laid the foundation for our university agent moving forward.

## Wrap Up

Great job! You’ve now seen how powerful well-crafted system instructions can be and how much subtle changes can reshape your agent’s behavior. Just as importantly, you’ve also seen that system prompts are more art than science: they evolve as you test the system, discover edge cases, and expand its capabilities.

More specifically, in this notebook, we:

- Defined the agent’s identity and role so it knows who it is and what domain it operates in.
- Clarified its capabilities by specifying what it can access and what it can help with.
- Added behavioral guidelines to shape tone, interaction style, and how it handles ambiguous or out-of-scope queries.
- Introduced constraints so the agent explicitly knows what it should *not* do.

Remember, this is just a starting point. System instructions can grow to include many more elements, such as:

- Example interactions (“when the user says X, respond like Y”)  
- Tool descriptions and usage rules (“you have access to tools A, B, C, and should use them when…”)  
- Fallback strategies and escalation paths for when the agent is unsure or underpowered  

As you iterate, treat your system prompt as a living document: refine it, extend it, and let your experiments guide its evolution.

Before moving on to the next section, use the code block below as your playground. You'll find some sample experiments to try in the dropdown below.

<details>
  <summary>🔶 Click the dropdown to see sample experiments to run.</summary>
  <br>

Experiment 1: Switch the advisor’s persona and tone

Goal: See how changing the identity and tone instructions affects the style of answers, while keeping the content mostly the same.

System prompt change:
- In the identity section, first set the agent to be a “formal academic advisor at Redis University who is precise and concise.”
- After testing, change it to “friendly peer mentor at Redis University who is encouraging, conversational, and uses simple language.”

Example queries:
1. Who are you, and how can you help me with course selection?  
2. Please explain how you will assist me in planning my learning for the next semester.

Compare how the answers differ between the two versions of the system prompt.

---

Experiment 2: Personalized plan for a sample student (Olivia)

Goal: Test how well the agent uses a detailed profile when the system prompt explicitly tells it to ground recommendations in that profile.

System prompt change:
- Add a section such as:
  - “When given a student profile, restate the key parts (background, interests, preferences, goals) and then map each course recommendation back to those details.”
  - “Prefer courses that align with the student’s long-term goals, even if they are slightly more challenging, but explain any trade-offs.”

Example query:

"Below is a student profile. Based on this information, please recommend 2–4 courses, in order, and briefly justify each recommendation in terms of the profile's background, interests, preferences, and goal."
   
> - Name: Olivia Jansen  
> - Major: Computer Science  
> - Year: Sophomore  
> - Completed: Intro to Programming (CS101), Data Structures (CS201), Calculus I  
> - Interests: Machine learning, Data science  
> - Preferences: Prefers online courses, learns best with hands-on projects  
> - Goal: Build a career in AI

Try removing or weakening that “profile” section in the system prompt and see how the recommendations and explanations change.

---

Experiment 3: Handling conflicting requirements and trade-offs

Goal: Make the agent explicitly reason about trade-offs when the student’s constraints cannot all be satisfied.

System prompt change:
- In the behavioral guidelines, add:
  - “If the student’s requirements conflict (for example, wanting something very advanced, very short, and beginner-friendly), explain the trade-offs clearly.”
  - “Offer the closest available option and be transparent about what compromises are being made.”

Example queries:
1. I want a course that is extremely advanced and requires no prior programming experience. Recommend the best fit and clearly explain which of my requirements you can and cannot satisfy.
2. I want an intermediate-level Redis course that is not too long and does not assume deep math knowledge. What would you recommend?

Verify whether the agent follows the new instructions to discuss trade-offs.

---

Experiment 4: Tool and capability honesty

Goal: Ensure the agent clearly communicates what it can and cannot do (for example, no direct enrollment, no real-time catalog access).

System prompt change:
- Add a capabilities and constraints block, such as:
  - “You do not have direct access to enrollment systems, student accounts, or live scheduling data.”
  - “When the user asks you to perform actions you cannot do, explain your limitations and suggest what they can do instead (for example, visiting a specific site or contacting support).”

Example queries:

1. Tell me which courses are open for enrollment next month, and sign me up for the best one for me.
2. Can you update my account so that it tracks my progress automatically?

Observe whether the agent now reliably explains its limits instead of pretending to have capabilities it does not have.


</details>
 <br>

In [ ]:
# Experiment with the system prompt below

custom_prompt = """You are the Redis University Class Agent, powered by Redis and the Agent Memory Server.

Your role is to help students:
- Find courses that match their interests and requirements
- Plan their academic schedule for the upcoming semesters
- Check prerequisites and course eligibility
- Get personalized course recommendations based on their goals

You have access to:
- A complete course catalog with descriptions, prerequisites, and schedules
- Student preferences and goals (stored in long-term memory)
- Conversation history (stored in working memory)
- Tools to search courses and check prerequisites

Guidelines:
- Be helpful, friendly, and encouraging
- Ask clarifying questions when you need more information
- Provide specific course recommendations with course codes and details
- Explain prerequisites and requirements clearly
- Remember student preferences and reference them in future conversations
- Stay focused on course planning and scheduling
- If asked about topics outside your domain, politely redirect to course planning

Example interactions:
- Student: "I'm interested in machine learning"
  You: "Great! I can help you find ML courses. What's your current year and have you taken any programming courses?"

- Student: "What are the prerequisites for CS401?"
  You: "Let me check that for you." [Use check_prerequisites tool]
"""

# Override agent instructions
set_system_instructions(custom_prompt)

queries = [
    {
        "label": "identity",
        "description": "Test the system's identity",
        "query": "Who are you and what can you do?"
    },
    {
        "label": "course recommendations",
        "description": "Test the courses it will recommend",
        "query": "Give me a recommendation on a course I should take next semester"
    },
    {
        "label": "off-topic query",
        "description": "Test an off-topic query",
        "query": "Suggest me a pizza recipe for my class"
    },
    {
        "label": "specific query",
        "description": "Test a specific query",
        "query": "I want to learn about databases and am not sure where to start"
    },
]

for q in queries:
    print(f"{q['description']}...")
    print(f"Running agent query on {q['label']}...")

    result = await workflow.ainvoke({"query": q["query"]})

    print(f"Response on {q['label']}:")
    print(result["final_answer"])
    print("\n" + "=" * 80 + "\n")